In [1]:
# Loading packages

import numpy as np 
import pandas as pd 
import json

In [2]:
# Loading data

train_data = pd.read_csv('../input/tweet-sentiment-extraction/train.csv')
test_data = pd.read_csv('../input/tweet-sentiment-extraction/test.csv')
submission_data = pd.read_csv('../input/tweet-sentiment-extraction/sample_submission.csv')

train = np.array(train_data)
test = np.array(test_data)

# Creating directory for data

!mkdir -p data

In [3]:
train_data.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


### Prepare data in QA format

Example-format:

```
train_data = [
    {
        'context': "This tweet sentiment extraction challenge is great",
        'qas': [
            {
                'id': "00001",
                'question': "positive",
                'answers': [
                    {
                        'text': "is great",
                        'answer_start': 43
                    }
                ]
            }
        ]
    }
    ]
```

In [4]:
%%time

# Source: https://www.kaggle.com/cheongwoongkang/roberta-baseline-starter-simple-postprocessing

def find_all(input_str, search_str):
    l1 = []
    length = len(input_str)
    index = 0
    while index < length:
        i = input_str.find(search_str, index)
        if i == -1:
            return l1
        l1.append(i)
        index = i + 1
    return l1

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 10.3 µs


In [5]:
# Source: https://www.kaggle.com/cheongwoongkang/roberta-baseline-starter-simple-postprocessing

# Prepare training data in QA-compatible format

def do_qa_train(train):

    output = []
    for line in train:
        context = line[1]

        qas = []
        question = line[-1]
        qid = line[0]
        answers = []
        answer = line[2]
        if type(answer) != str or type(context) != str or type(question) != str:
            print(context, type(context))
            print(answer, type(answer))
            print(question, type(question))
            continue
        answer_starts = find_all(context, answer)
        for answer_start in answer_starts:
            answers.append({'answer_start': answer_start, 'text': answer.lower()})
            break
        qas.append({'question': question, 'id': qid, 'is_impossible': False, 'answers': answers})

        output.append({'context': context.lower(), 'qas': qas})
        
    return output

qa_train = do_qa_train(train)

with open('data/train.json', 'w') as outfile:
    json.dump(qa_train, outfile)

nan <class 'float'>
nan <class 'float'>
neutral <class 'str'>


In [6]:
%%time

# Prepare testing data in QA-compatible format

def do_qa_test(test):
    output = []
    for line in test:
        context = line[1]
        qas = []
        question = line[-1]
        qid = line[0]
        if type(context) != str or type(question) != str:
            print(context, type(context))
            print(answer, type(answer))
            print(question, type(question))
            continue
        answers = []
        answers.append({'answer_start': 1000000, 'text': '__None__'})
        qas.append({'question': question, 'id': qid, 'is_impossible': False, 'answers': answers})
        output.append({'context': context.lower(), 'qas': qas})
    return output

qa_test = do_qa_test(test)

with open('data/test.json', 'w') as outfile:
    json.dump(qa_test, outfile)

CPU times: user 132 ms, sys: 1.72 ms, total: 134 ms
Wall time: 133 ms


In [7]:
# Installing simple-transformers tool to test our pre-trained model

!pip install '/kaggle/input/simple-transformers-pypi/seqeval-0.0.12-py3-none-any.whl' -q
!pip install '/kaggle/input/simple-transformers-pypi/simpletransformers-0.22.1-py3-none-any.whl' -q

In [8]:
%%time

# Training our DistilBERT model (distilbert-base-uncased-distilled-squad)

from simpletransformers.question_answering import QuestionAnsweringModel

MODEL_PATH = '../input/transformers-pretrained-distilbert/distilbert-base-uncased-distilled-squad/'

model_args = {'reprocess_input_data': True,
              'overwrite_output_dir': True,
              'learning_rate': 5e-5,
              'num_train_epochs': 2,
              'max_seq_length': 192,
              'doc_stride': 64,
              'fp16': False
             }

# Create the QuestionAnsweringModel

model = QuestionAnsweringModel('distilbert', 
                               MODEL_PATH, 
                               args = model_args,
                               use_cuda = True)

model.train_model('data/train.json')

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils.py:831: FutureWarning: Parameter max_len is deprecated and will be removed in a future release. Use model_max_length instead.
  category=FutureWarning,
100%|██████████| 27480/27480 [00:40<00:00, 686.87it/s]


Running loss: 3.462543

/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 0.641953


Running loss: 0.281324

CPU times: user 9min 18s, sys: 2min 43s, total: 12min 1s
Wall time: 12min 9s


In [9]:
%%time

# Submission

predictions = model.predict(qa_test)
predictions_df = pd.DataFrame.from_dict(predictions)

submission_data['selected_text'] = predictions_df['answer']

submission_data.to_csv('submission.csv', index = False)

print("File submitted successfully.")

100%|██████████| 3534/3534 [00:04<00:00, 794.45it/s]



File submitted successfully.
CPU times: user 29.9 s, sys: 3.37 s, total: 33.3 s
Wall time: 33.6 s
